目前不依赖于原有项目，独立完成推特用户意见分类器

# 创造训练数据

In [158]:
# 聚合所有数据
import glob
import pandas as pd

X = []
y = []
for in_file in glob.glob("data/tokens.train/*.txt"):
    print(in_file)
    for line in open(in_file):
        label, sentense = line.strip().split("\t")
        if label == "0":
            label = "FF"
        elif label == "1":
            label = "MP"
        else:
            assert "Error"
        X.append(sentense)
        y.append(label)

data = {}
data["label"] = y
data["text"] = X
pd.DataFrame(data).to_csv("data/train.csv", index=None)

data/tokens.train/tokens.08.txt
data/tokens.train/tokens.05.txt
data/tokens.train/tokens.04.txt
data/tokens.train/tokens.06.txt
data/tokens.train/tokens.07.txt


In [1]:
import torchtext

TEXT = torchtext.data.Field()
LABEL = torchtext.data.LabelField()

my_data = torchtext.data.TabularDataset(path='data/train.csv',
                                        format='csv', 
                                        fields=[('labels', LABEL), ('text', TEXT)])

# train_data, valid_data = my_data.split(split_ratio=0.8)
# train_data, valid_data

In [2]:
TEXT.build_vocab(my_data, min_freq=10)
LABEL.build_vocab(my_data)

In [3]:
len(TEXT.vocab), len(LABEL.vocab)

(21142, 3)

In [4]:
LABEL.vocab.stoi

defaultdict(None, {'MP': 0, 'FF': 1, 'label': 2})

In [5]:
train_data, valid_data = my_data.split(split_ratio=0.8)
# train_data, valid_data

train_iter, valid_iter = torchtext.data.iterator.BucketIterator.splits((train_data, valid_data),
                                             batch_sizes=(32, 32),
                                             sort_key=lambda x: len(x.text), # field sorted by len
                                             sort_within_batch=True,
                                             repeat=False)

In [6]:
batch = next(iter(train_iter))
print(train_iter)
print('batch:\n', batch)
print('batch_text:\n', batch.text)
print('batch_label:\n', batch.label)

batch:
 
[torchtext.data.batch.Batch of size 32]
	[.labels]:[torch.LongTensor of size 32]
	[.text]:[torch.LongTensor of size 21x32]
batch_text:
 tensor([[    0,    13,     0,     2,   789,   253,   547,  1019,   128,     2,
           303,  4281,     2,     2,   381,  9177,    20,    11,   151,  7175,
             2,     2,    83,    39,     0, 13573,    11,    52,     2,     6,
          1944,   489],
        [    0,   297,   298,     6,     8,  1988,   115,    25,    29,    39,
             2,     0,    34, 11142,   108,   268,     4,   931,  1118,     0,
            20,    20,    17,   989,    40,  6441,   121,   169,    20,   809,
            28,    13],
        [    0,  1599,   298,   421,    41,     3,  9113,   492,  5066,  1100,
            18,   961,    13,    20,    10,     0,   102, 13465,    13,     9,
             0,    17,    11,    13,     0,    92,    21,  5832,    21,     3,
             2,    11],
        [    0,     4,    10,     8, 18508,     8,     4, 11322,    12, 

AttributeError: 'Batch' object has no attribute 'label'

# raw word to ix

In [ ]:
# def prepare_sequence(seq, to_ix):
#     idxs = [to_ix[w] for w in seq]
#     return torch.tensor(idxs, dtype=torch.long)


# training_data = [
#     ("The dog ate the apple".split(), ["DET", "NN", "V", "DET", "NN"]),
#     ("Everybody read that book".split(), ["NN", "V", "DET", "NN"])
# ]
# word_to_ix = {}
# for sent, tags in training_data:
#     for word in sent:
#         if word not in word_to_ix:
#             word_to_ix[word] = len(word_to_ix)
# print(word_to_ix)
# tag_to_ix = {"DET": 0, "NN": 1, "V": 2}

# # These will usually be more like 32 or 64 dimensional.
# # We will keep them small, so we can see how the weights change as we train.
# EMBEDDING_DIM = 6
# HIDDEN_DIM = 6

# 模型

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F

torch.manual_seed(23)


class TweetClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

In [9]:
VOCAB_SIZE = len(TEXT.vocab)
EMBED_DIM = 300
NUN_CLASS = 2
model = TweetClassifier(VOCAB_SIZE, EMBED_DIM, NUN_CLASS)

# 训练

In [ ]:
def train_func(sub_train_):

    # Train the model
    train_loss = 0
    train_acc = 0
    # data = DataLoader(sub_train_, batch_size=BATCH_SIZE, shuffle=True,
    #                   collate_fn=generate_batch)
    # for i, (text, offsets, cls) in enumerate(data):
    #     optimizer.zero_grad()
    #     text, offsets, cls = text.to(device), offsets.to(device), cls.to(device)
    #     output = model(text, offsets)
    #     loss = criterion(output, cls)
    #     train_loss += loss.item()
    #     loss.backward()
    #     optimizer.step()
    #     train_acc += (output.argmax(1) == cls).sum().item()
    for batch in train_iter:
        optimizer.zero_grad()
        text, offsets, cls = text.to(device), offsets.to(device), cls.to(device)
        output = model(batch)
        loss = criterion(output, cls)
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
        train_acc += (output.argmax(1) == cls).sum().item()    

    # Adjust the learning rate
    scheduler.step()

    return train_loss / len(sub_train_), train_acc / len(sub_train_)

def test(data_):
    loss = 0
    acc = 0
    data = DataLoader(data_, batch_size=BATCH_SIZE, collate_fn=generate_batch)
    for text, offsets, cls in data:
        text, offsets, cls = text.to(device), offsets.to(device), cls.to(device)
        with torch.no_grad():
            output = model(text, offsets)
            loss = criterion(output, cls)
            loss += loss.item()
            acc += (output.argmax(1) == cls).sum().item()

    return loss / len(data_), acc / len(data_)

# 验证